# Creating an Amundsen Sea Low (ASL) index

This work is a continuation of my 2013 and 2016 papers as described [here](https://scotthosking.com/asl_index)

## Read in data and run the calculations

In [2]:
from asli.asli import ASLICalculator
a = ASLICalculator(data_dir="../data/", 
                   mask_filename="era5_lsm.nc",
                   msl_pattern="ERA5/monthly/era5_mean_sea_level_pressure_monthly_*.nc"
                   )
a.read_mask_data()
a.read_msl_data()

In [4]:
a.calculate()

,time,lon,lat,ActCenPres,SectorPres,RelCenPres
0,1979-01-01,220.25,-68.25,982.861328,986.142883,-3.281555
1,1979-02-01,195.75,-71.50,973.644226,982.679382,-9.035156
2,1979-03-01,223.50,-69.75,972.363342,980.153687,-7.790344
3,1979-04-01,270.00,-67.50,967.868103,979.046631,-11.178528
4,1979-05-01,190.25,-70.50,977.977905,987.413574,-9.435669
5,1979-06-01,213.50,-70.50,968.613647,978.659058,-10.045410
6,1979-07-01,212.00,-74.75,973.736023,980.764771,-7.028748
7,1979-08-01,256.50,-72.00,972.256042,987.383240,-15.127197
8,1979-09-01,207.00,-74.75,973.641968,982.257385,-8.615417
9,1979-10-01,251.25,-72.00,972.795776,982.633118,-9.837341


## Select region and apply land-sea mask

In [ ]:
da_mask = da.where(mask == 0)

# region of interest (asl sector)
asl_region = {'west':170., 'east':298., 'south':-80., 'north':-60.}

### slice area around ASL region
da      = slice_region(da, asl_region)
da_mask = slice_region(da_mask, asl_region)

In [ ]:
plt.figure(figsize=(5,5))
ax1 = plt.subplot( 121, projection=ccrs.Stereographic(central_longitude=0., central_latitude=-90.) )
ax1.set_extent([-180,180,-90,-50], ccrs.PlateCarree())
da.isel(time=-1).plot.pcolormesh( 'longitude', 'latitude', cmap='Reds', 
                                        transform=ccrs.PlateCarree(), add_colorbar=False )

ax2 = plt.subplot( 122, projection=ccrs.Stereographic(central_longitude=0., central_latitude=-90.) )
ax2.set_extent([-180,180,-90,-50], ccrs.PlateCarree())
da_mask.isel(time=-1).plot.pcolormesh( 'longitude', 'latitude', cmap='Reds', 
                                        transform=ccrs.PlateCarree(), add_colorbar=False )

## Loop through each time and identify lows

## Plotting: location of minimas in pressure field

In [ ]:
def draw_regional_box( region, transform=None ):
    '''
    Draw box around a region on a map
    region is a dictionary with west,east,south,north
    '''

    if transform == None:
        transform = ccrs.PlateCarree()

    plt.plot([region['west'], region['west']], [region['south'],region['north']], 
                 'k-', transform=transform, linewidth=1)
    plt.plot([region['east'], region['east']], [region['south'],region['north']], 
                 'k-', transform=transform, linewidth=1)
    
    for i in range( int(region['west']),int(region['east']) ): 
        plt.plot([i,i+1], [region['south'],region['south']], 'k-', transform=transform, linewidth=1)
        plt.plot([i,i+1], [region['north'],region['north']], 'k-', transform=transform, linewidth=1)

In [ ]:
plt.figure(figsize=(20,15))

for i in range(0,12):
    
    da_2D = da_mask.isel(time=i)

    da_2D = da_2D.sel(latitude=slice(-55,-90),longitude=slice(165,305))

    ax = plt.subplot( 3, 4, i+1, 
                        projection=ccrs.Stereographic(central_longitude=0., 
                                                      central_latitude=-90.) )

    ax.set_extent([165,305,-85,-55], ccrs.PlateCarree())

    result = da_2D.plot.contourf( 'longitude', 'latitude', cmap='Reds', 
                                    transform=ccrs.PlateCarree(), 
                                    add_colorbar=False, 
                                    levels=np.linspace(np.nanmin(da_2D.values), np.nanmax(da_2D.values), 20) )

    # ax.coastlines(resolution='110m')
    ax.set_title(str(da_2D.time.values)[0:7])

    ## mark ASL
    df2 = asl_df[ asl_df['time'] == str(da_2D.time.values)[0:10]]
    if len(df2) > 0:
        ax.plot(df2['lon'], df2['lat'], 'mx', transform=ccrs.PlateCarree() )
    draw_regional_box(asl_region)

print('')